### Connect to workspace

In [14]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

ws = Workspace.from_config(path="config.json")

try:
    ct = ComputeTarget(ws, "ml-vm")
except ComputeTargetException as e:
    print(e)

print(ws)

Workspace.create(name='iot-ml-model', subscription_id='910339dc-1f1f-43d9-b38f-1594badf708f', resource_group='azure-iot')


### Create Environmet

In [15]:
env = Environment.from_pip_requirements("model-iot-cpu-env", "ml_model_requirements.txt")
env.register(ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20211029.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "model-iot-cpu-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-for

### Create Experiment

In [16]:
exp = Experiment(ws, "linear-regression-1")

# set up script run configuration
config = ScriptRunConfig(
    source_directory='.',
    command='pip install azureml-sdk && python train.py --config config.yaml'.split(),
    compute_target=ct,
    environment=env,
    #arguments=['--config', "config.yaml"],
)

# submit script to AML
run = exp.submit(config)
print(run.get_portal_url()) # link to ml.azure.com
run.wait_for_completion(show_output=True)


https://ml.azure.com/runs/linear-regression-1_1638479757_cc351787?wsid=/subscriptions/910339dc-1f1f-43d9-b38f-1594badf708f/resourcegroups/azure-iot/workspaces/iot-ml-model&tid=94c16db7-8eba-4986-aac3-66f74bfafc9e
RunId: linear-regression-1_1638479757_cc351787
Web View: https://ml.azure.com/runs/linear-regression-1_1638479757_cc351787?wsid=/subscriptions/910339dc-1f1f-43d9-b38f-1594badf708f/resourcegroups/azure-iot/workspaces/iot-ml-model&tid=94c16db7-8eba-4986-aac3-66f74bfafc9e

Streaming azureml-logs/55_azureml-execution-tvmps_dfdc7f89782f1f12ba43684a0d6355d3cf287e186a14766f661e5bc026c814c0_d.txt

2021-12-02T21:16:11Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/iot-ml-model/azureml/linear-regression-1_1638479757_cc351787/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/iot-ml-model/azureml/linear-regression-1_1638479757_cc351787/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-m

{'runId': 'linear-regression-1_1638479757_cc351787',
 'target': 'ml-vm',
 'status': 'Completed',
 'startTimeUtc': '2021-12-02T21:16:11.117571Z',
 'endTimeUtc': '2021-12-02T21:17:18.043138Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '7cce6776-a3a6-41c3-b99c-642097544c01',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': None,
  'command': 'pip install azureml-sdk && python train.py --config config.yaml',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'ml-vm',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity

In [17]:
model = run.register_model(model_name='iot-linear-model', model_path='outputs/models/model.pt')